In [184]:
from typing import Dict, List
from abc import ABC
import chardet
import json
import pandas as pd
from datetime import datetime
import glob, os

class AbcFileParser(ABC):
    """
    path - путь к директории c источниками
    struc - ключи и их индекс
    """
    def __init__(self, path:str,  enc:str, struc:Dict[int,str]):
        pass
    
    def read(self):
        pass
    
    def to_df(self):
        pass
    
    #удаляем лишние столбцы. dict объект в перспективе может давать информацию о индексе столбца
    def cut_df(self):
        for i in self.df.columns:
            if i not in self.struc.values():
                self.df = self.df.drop([i], axis=1)
            else:
                pass
            
    def calc_total(self, price_column:str='Цена, ед.', quantity_column:str='Кол-во, шт'):
        self.df['total_count'] = a.df[price_column] * a.df[quantity_column]
    
    
class JsonFileParser(AbcFileParser):
    def __init__(self, path:str,  enc:str, struc: Dict[int,str]):
        super().__init__(path=path, enc=enc, struc=struc)
        self.path = path
        self.enc = enc
        self.struc = struc
        self.hub_name = path.replace('./','').replace(' ','')
  
    def read(self):
        with open(self.path, 'r', encoding=self.enc) as file:
            self.data = json.loads(file.read())
            
    def to_df(self):
        self.read()
        self.df = pd.read_json(json.dumps(self.data))
        
a = JsonFileParser('./Екатеринбург_июнь.json',
                   'utf-8',
                  {1:'Код товара 2',
                    2:'Товар',
                    3:'Цена, ед.',
                    4:'Кол-во, шт',
                    5:'Времени до окончания срока годности, %',
                    6:'Отпустил товар'
                   }
                  )

a.to_df()
a.cut_df()
a.calc_total()
a.df

,Код товара 2,Товар,"Цена, ед.","Кол-во, шт",Отпустил товар,total_count
0,F371461,"Биойогурт питьевой обогащ. ""Тёма"" Чернослив 2,...",489,90,Мальцев Александр Егорович,44010
1,F369125,"Вермишель б/п MIVIMEX со вкусом свинины, 50 гр.",389,370,Соколов Степан Тимофеевич,143930
2,F541553,Погрузка БВК,221,750,Павлов Вячеслав Матвеевич,165750
3,F460837,Молоко сг.Машенька (Саранск) вареное 380г,230,160,Галкин Василий Артёмович,36800
4,F385725,Майонез Mr.Ricco Пер.яйцо Organic 67% 800мл (д...,213,190,Семенов Максим Артёмович,40470
...,...,...,...,...,...,...
127,F163514,Grill Mates Копченый техасец Kamis 20г (фольг....,319,540,Галкин Василий Артёмович,172260
128,F150979,"0659-36 Чай Принцесса Гита лесные ягоды 1,5гр*...",417,30,Мальцев Александр Егорович,12510
129,F280361,"Питьевой биойгурт ""Био-Баланс"" гранат 1,5% 330...",275,710,Соколов Давид Михайлович,195250
130,F660998,"Рис шлифованный пропаренный длинный ""ГУДВИЛЛ""...",295,150,Попова Ольга Никитична,44250


In [164]:
class CsvFileParser(AbcFileParser):
    def __init__(self, path:str,  enc:str, struc: Dict[int,str]):
        super().__init__(path=path, enc=enc, struc=struc)
        self.path = path
        self.enc = enc
        self.struc = struc
        self.hub_name = path.replace('./','').replace(' ','')

    def to_df(self):
        self.df = pd.read_csv(self.path)
        
b = CsvFileParser('Нижний_Новгород_июнь.csv',
                  'utf-8',
                  {1:'Код товара 2',
                    2:'Товар',
                    3:'Цена, ед.',
                    4:'Кол-во, шт',
                    5:'Времени до окончания срока годности, %',
                    6:'Отпустил товар'
                   }
                 )
b.to_df()
b.cut_df()
b.calc_total()
b.df


,Код товара 2,Товар,"Цена, ед.","Кол-во, шт","Времени до окончания срока годности, %",total_count
0,F168128,65414916Наваристый говяжий бульон Кнорр 2*28г ...,167,600,0.8,44010.0
1,F369779,КвасБукет Чувашии Зимний натур. брожения 0.5л...,185,260,0.5,143930.0
2,F547346,"Гвоздика молотая ТМ""Мастер Дак"" 10гр",321,200,0.9,165750.0
3,F772852,Универ. смесь пряностей для запекания птицы ТМ...,200,560,1.0,36800.0
4,F110363,"65415123Суп ""Чашка супа"" Кнорр 14гр Мясной с л...",256,990,1.0,40470.0
...,...,...,...,...,...,...
131,F765538,1291 Чай Ahmad Tea Цейлонский чай F.B.O.P.F ка...,122,310,0.9,249150.0
132,F860049,Сыр Эдам Джагибо,254,770,0.9,NaN
133,F763570,"115611Чай Майский Шахразада, черный, 25*2г",308,790,0.8,NaN
134,F258134,Картофельное пюре б/п MIVIMEX с луком игренкам...,483,360,0.6,NaN


In [165]:
class XlsxFileParser(AbcFileParser):
    def __init__(self, path:str,  enc:str, struc: Dict[int,str], sheet_name:str='Sheet1', engine:str='openpyxl'):
        super().__init__(path=path, enc=enc, struc=struc)
        self.path = path
        self.enc = enc
        self.struc = struc
        self.sheet_name = sheet_name
        self.engine = engine
        self.hub_name = path.replace('./','').replace(' ','')

    def to_df(self):
        self.df = pd.read_excel(self.path, sheet_name=self.sheet_name, engine=self.engine)
        
a = XlsxFileParser('Москва_апрель.xlsx',
                   'utf-8',
                   {1:'Код товара 1',
                    2:'Товар',
                    3:'Цена, ед.',
                    4:'Кол-во, шт',
                    5:'Времени до окончания срока годности, %',
                    6:'Отпустил товар'
                   })

a.to_df()
a.cut_df()
a.calc_total()
a.df


,Код товара 1,Товар,"Цена, ед.","Кол-во, шт","Времени до окончания срока годности, %",Отпустил товар,total_count
0,25318,Розмарин Kamis 10г (фольг. пакет),142,80,0.6,Гусев Виталий Ибрагимович,11360
1,23796,751 Хлопья Увелка овсяные традиционные 500гр,146,680,0.9,Дементьев Иван Дмитриевич,99280
2,24773,"Рожок суповой ""Сытое царство"" в/с 900гр",286,60,0.8,Титов Роберт Егорович,17160
3,24894,"21073964Суп ""Чашка супа"" Кнорр 15гр Шпинатный ...",275,890,0.9,Орлов Ярослав Тихонович,244750
4,25616,"Холодильное оборудование FV-500 ""Моя семья"" се...",196,400,0.9,Мальцев Александр Егорович,78400
...,...,...,...,...,...,...,...
715,27353,"л-146 Русский аппетит ""Перец черный горошек"" 1...",149,700,0.7,Соколов Степан Тимофеевич,104300
716,22718,Хлопья Любятово мультизерновые с изюмом и клюк...,246,540,0.5,Гусев Петр Михайлович,132840
717,27185,512708Чай Curtis Truffle Tea (Трюффле блек Ти)...,438,540,0.6,Гусев Виталий Ибрагимович,236520
718,26605,"Картофельное пюре б/п ""Биг Ланч"" ""КГ"", с гуляш...",113,800,0.9,Семенов Максим Артёмович,90400


In [166]:
a.df['total_count'].sum()
a.hub_name

'Москва_апрель.xlsx'

In [238]:
class Counter:
    def __init__(self, objs:List[AbcFileParser]):
        self.objs = objs

    def create(self):
        data = []
        for df in self.objs:
            count = df.df['total_count'].sum()
            quantity = df.df['Кол-во, шт'].sum()
            data.append((df.hub_name, quantity, count))
        self.data = data
    
    def to_xlsx(self, path:str='./', sheet_name:str='repopt') -> None:
        df_data = {}
        df_data['Склад'] = [i[0] for i in self.data]
        df_data['Кол-во, шт'] = [i[1] for i in self.data]
        df_data['Стоимость, руб.'] = [i[2] for i in self.data]
        
        new_df = pd.DataFrame(data = df_data)
        new_df.to_excel("report.xlsx",
             sheet_name=sheet_name)
        return None

c = Counter([a,b])
c.create()
c.to_xlsx()

            

In [196]:
class CheckDate:
    
    def __init__(self):
        date_map = {
            1:'январь', 2:'февраль', 3:'март', 4:'апрель',
            5:'май', 6:'июнь', 7:'июль', 8:'август',
            9:'сентябрь', 10:'октябрь', 11:'ноябрь', 12:'декабрь'
        }
        self.date = date_map.get(datetime.now().month)
        

cc = CheckDate()
cc.date

'август'

In [183]:
cc.date.find('стув')

-1

In [218]:
class DocFinder:
    
    def __init__(self, a_date:str, path_dir:str = './'):
        self.a_date = a_date
        self.cur_dir = os.getcwd()
        self.path_dir = path_dir
        self.csv = []
        self.json = []
        self.xlsx = []
        
    def get_files(self):
        os.chdir(self.path_dir)
        for file in glob.glob("*.csv"):
            if file.find(self.a_date) != -1:
                self.csv.append(file)
                
        for file in glob.glob("*.json"):
            if file.find(self.a_date) != -1:
                self.json.append(file)
                
        for file in glob.glob("*.xlsx"):
            if file.find(self.a_date) != -1:
                self.xlsx.append(file)
        
        
dd = DocFinder(cc.date)
dd.get_files()

In [243]:
class Pipe:
    
    def __init__(self, srcs:DocFinder, strucs: Dict[int,Dict[int,str]] = None):
        #strucs может использоваться для конфигурирования FileParser
        #однаком имхо выходит за рамки тестовой работы
        #в реальности оговаривается и описывается в доке, арх. данных и постановках
        self.srcs = srcs
        self.dfs = []
        self.strucs = strucs
    
    def to_df(self):
        for i in self.srcs.csv:
            try:
                self.dfs.append(CsvFileParser(i,
                  'utf-8',
                  {1:'Код товара 2',
                    2:'Товар',
                    3:'Цена, ед.',
                    4:'Кол-во, шт',
                    5:'Времени до окончания срока годности, %',
                    6:'Отпустил товар'
                   }
                 )
            )
            except Exception as ex:
                print(ex)
        for i in self.srcs.json:
            try:
                self.dfs.append(JsonFileParser(i,
                  'utf-8',
                  {1:'Код товара 2',
                    2:'Товар',
                    3:'Цена, ед.',
                    4:'Кол-во, шт',
                    5:'Времени до окончания срока годности, %',
                    6:'Отпустил товар'
                   }
                 )
            )
            except Exception as ex:
                print(ex)
                
        for i in self.srcs.xlsx:
            try:
                self.dfs.append(XlsxFileParser(i,
                  'utf-8',
                  {1:'Код товара 1',
                    2:'Товар',
                    3:'Цена, ед.',
                    4:'Кол-во, шт',
                    5:'Времени до окончания срока годности, %',
                    6:'Отпустил товар'
                   }
                 )
            )
            except Exception as ex:
                print(ex)
                
        for i in self.dfs:
            i.to_df()
        
    def to_clean(self):
        for i in self.dfs:
            i.cut_df()
        
    def to_calculate(self):
        for i in self.dfs:
            i.calc_total()
                
    def unite(self):
        self.united = Counter(self.dfs)
        self.united.create()
            
    def save(self):
        self.united.to_xlsx()
            

pp = Pipe(dd)
pp.to_df()
pp.dfs

In [244]:
pp.to_clean()
pp.to_calculate()
pp.unite()
pp.save()


,Код товара 2,Товар,"Цена, ед.","Кол-во, шт","Времени до окончания срока годности, %"
0,F168128,65414916Наваристый говяжий бульон Кнорр 2*28г ...,167,600,0.8
1,F369779,КвасБукет Чувашии Зимний натур. брожения 0.5л...,185,260,0.5
2,F547346,"Гвоздика молотая ТМ""Мастер Дак"" 10гр",321,200,0.9
3,F772852,Универ. смесь пряностей для запекания птицы ТМ...,200,560,1.0
4,F110363,"65415123Суп ""Чашка супа"" Кнорр 14гр Мясной с л...",256,990,1.0
...,...,...,...,...,...
131,F765538,1291 Чай Ahmad Tea Цейлонский чай F.B.O.P.F ка...,122,310,0.9
132,F860049,Сыр Эдам Джагибо,254,770,0.9
133,F763570,"115611Чай Майский Шахразада, черный, 25*2г",308,790,0.8
134,F258134,Картофельное пюре б/п MIVIMEX с луком игренкам...,483,360,0.6
